In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd ../TileEM/

/home/jlee782/crowd-seg/analysis/TileEM


In [25]:
import os 
import pickle as pkl 
from  analysis_toolbox import * 

# Looking at best T' from sampled set of workers

In [18]:
objid=1

In [22]:
os.chdir("sample/"+dir_name)
workers=pkl.load(open("worker{}.pkl".format(objid)))
tiles = pkl.load(open("vtiles{}.pkl".format(objid)))
os.chdir("../..")

In [27]:
print workers

[187, 156, 159, 248, 170, 223, 140, 247, 196, 184, 137, 133, 175, 52, 219, 131, 246, 114, 116, 148, 182, 201, 208, 149, 239, 118, 181, 146, 238, 234]


In [29]:
from TileEM import *

In [33]:
from BB2TileExact import *

In [65]:
def overlap(a,b,mode='smaller'):
    if mode=="larger":
        if a.area>b.area:
            norm_area= a.area
        else:
            norm_area = b.area
    elif mode=="smaller":
        if a.area<b.area:
            norm_area= a.area
        else:
            norm_area = b.area
    try:    
        return a.intersection(b).area/norm_area
    except(shapely.geos.TopologicalError):
        return a.buffer(1e-10).intersection(b).area/norm_area

In [70]:
overlap(tk,T,mode='larger')

0.0005464064325622983

In [68]:
overlap(tk,T)

0.0015959255911771437

In [87]:
cd ../../

/home/jlee782/crowd-seg/analysis/TileEM


In [83]:
my_BBG  = pd.read_csv("my_ground_truth.csv")

In [124]:
worker_Nbatches={5:10,10:8,15:6,20:4,25:2,30:1}
sampleN_lst=worker_Nbatches.keys()
for Nworker in sampleN_lst[:2]:
    for batch_num in range(worker_Nbatches[Nworker]):
        DATA_DIR = "sample/{0}worker_rand{1}/".format(Nworker,batch_num)
        Qj=[]
        best_P_lst = []
        best_R_lst = []
        A_thres_lst=[]
        df = pd.read_csv(DATA_DIR+"PR_tbl_all.csv",index_col=0)
        for objid in object_lst:
            ground_truth_match = my_BBG[my_BBG.object_id==objid]
            x_locs,y_locs =  process_raw_locs([ground_truth_match["x_locs"].iloc[0],ground_truth_match["y_locs"].iloc[0]])
            T = Polygon(zip(x_locs,y_locs))
            tiles = pkl.load(open(DATA_DIR+"vtiles{}.pkl".format(objid)))
            indMat = pkl.load(open(DATA_DIR+"indMat{}.pkl".format(objid)))
            workers = pkl.load(open(DATA_DIR+"/worker{}.pkl".format(objid)))
            bestTprime = []
            for i,tk in enumerate(tiles):
                if overlap(tk,T)>0.5:
                    bestTprime.append(i)
            if bestTprime==[]:
                P=0
                R=0
            else:
                P,R = compute_PR(objid,np.array(bestTprime),tiles)
            tile_area = np.array(indMat[-1])
            A_thres = np.median(tile_area[bestTprime])
            best_P_lst.append(P)
            best_R_lst.append(R)
            A_thres_lst.append(A_thres)
#             print P,R,A_thres
            for wid in workers:
                j = workers.index(wid)
                qn1,qn2,qp1,qp2 = QjGTLSA(tiles,indMat,T,j,A_thres)
                Qj.append([objid,wid,j,qn1,qn2,qp1,qp2])
        GTdf = pd.DataFrame(np.array([object_lst,best_P_lst,best_R_lst,A_thres_lst]).T,columns=["object_id","GT Tile-based Precision","GT Tile-based Recall","GT A_thres"])    
        GTdf.to_csv(DATA_DIR+"GT_PR.csv")
        Qj_tbl = pd.DataFrame(Qj,columns=["object_id","worker_id","j","Qn1","Qn2","Qp1","Qp2"])
        Qj_tbl.to_csv(DATA_DIR+"GT_Qj.csv")
#         break
#     break

TopologyException: found non-noded intersection between LINESTRING (306 219, 304.75 215.25) and LINESTRING (304.75 215.25, 305.333 217) at 304.94422743055554 215.83268229166663
TopologyException: found non-noded intersection between LINESTRING (314.5 150.5, 310 155) and LINESTRING (312 151, 310 155) at 310 154.99999999999994
Too few points in geometry component at or near point 328.25 147
TopologyException: found non-noded intersection between LINESTRING (238.609 423.261, 252 418) and LINESTRING (251 415, 238.609 423.261) at 238.60869565217391 423.26086956521743
TopologyException: found non-noded intersection between LINESTRING (226.732 427.927, 236.98 423.901) and LINESTRING (235.966 424.299, 226.732 427.927) at 230.82200575884968 426.31992630902329
TopologyException: side location conflict at 241.12389380530973 336.26991150442478
TopologyException: found non-noded intersection between LINESTRING (242.182 420.818, 242.182 420.818) and LINESTRING (242.182 420.818, 244.232 419.603) at 2

TopologyException: side location conflict at 118.83333333333333 398.58333333333331
TopologyException: found non-noded intersection between LINESTRING (131.312 443.688, 131.508 440.365) and LINESTRING (131.5 440.505, 131.508 440.365) at 131.49967805895449 440.50547299777372
TopologyException: found non-noded intersection between LINESTRING (86.1589 433.458, 86.2069 432.69) and LINESTRING (90 394, 85.6522 438.348) at 86.206896551724142 432.68965517241378
Self-intersection at or near point 118.83333333333333 398.58333333333331
TopologyException: found non-noded intersection between LINESTRING (86.0909 434.545, 86.2069 432.69) and LINESTRING (90 394, 85.6522 438.348) at 86.206896551724142 432.68965517241378
TopologyException: found non-noded intersection between LINESTRING (105 360, 98.3043 348.043) and LINESTRING (98.3043 348.043, 100 352) at 98.304347826086996 348.04347826086962
TopologyException: found non-noded intersection between LINESTRING (116 388.222, 109 368) and LINESTRING (110.

TopologyException: found non-noded intersection between LINESTRING (289.176 184.588, 295.438 180.623) and LINESTRING (295.438 180.623, 295.158 181.93) at 295.43801652892557 180.62258953168046
TopologyException: EdgeRing::computePoints: found null Directed Edge
TopologyException: found non-noded intersection between LINESTRING (270 197, 269.069 197.323) and LINESTRING (273.04 194.808, 269.069 197.323) at 269.06888361045145 197.32304038004744
TopologyException: found non-noded intersection between LINESTRING (490.182 220.545, 493.197 222.484) and LINESTRING (490.182 220.545, 495.636 224.909) at 490.18181818181836 220.54545454545467
TopologyException: found non-noded intersection between LINESTRING (466.444 222.889, 478 226) and LINESTRING (481 226, 469.333 223.667) at 469.33333333333331 223.66666666666666
TopologyException: found non-noded intersection between LINESTRING (490.182 220.545, 493.197 222.484) and LINESTRING (490.182 220.545, 495.636 224.909) at 490.18181818181836 220.5454545

Self-intersection at or near point 387.25274725274721 346.31868131868123
TopologyException: found non-noded intersection between LINESTRING (384.063 321.606, 383.35 318.515) and LINESTRING (384.063 321.606, 384.025 321.565) at 384.06299212598424 321.6062992125984
Self-intersection at or near point 387.25274725274721 346.31868131868123
TopologyException: found non-noded intersection between LINESTRING (386.4 344.4, 387.253 346.319) and LINESTRING (380 341, 387.253 346.319) at 387.25274725274721 346.31868131868123
Self-intersection at or near point 387.25274725274721 346.31868131868123
TopologyException: found non-noded intersection between LINESTRING (386.4 344.4, 387.253 346.319) and LINESTRING (380 341, 387.253 346.319) at 387.25274725274721 346.31868131868123
Self-intersection at or near point 387.25274725274721 346.31868131868123
TopologyException: found non-noded intersection between LINESTRING (386.4 344.4, 387.253 346.319) and LINESTRING (380 341, 387.253 346.319) at 387.25274725

TopologyException: found non-noded intersection between LINESTRING (481.584 200.384, 481.856 200.656) and LINESTRING (481.251 200.691, 482.683 200.607) at 481.8555670844184 200.65555487738715
TopologyException: found non-noded intersection between LINESTRING (495.508 213.377, 491.882 208.784) and LINESTRING (491 208, 492.25 209.25) at 492.25 209.25
TopologyException: found non-noded intersection between LINESTRING (469 212, 468.37 216.2) and LINESTRING (468.37 216.2, 473.16 208.16) at 468.36992840095468 216.20047732696892
TopologyException: found non-noded intersection between LINESTRING (569.407 243.255, 562.316 227.654) and LINESTRING (561.17 226.857, 565.253 234.116) at 565.25270412360192 234.11591844195905
TopologyException: found non-noded intersection between LINESTRING (481.584 200.384, 481.856 200.656) and LINESTRING (481.251 200.691, 482.683 200.607) at 481.8555670844184 200.65555487738715
TopologyException: found non-noded intersection between LINESTRING (495.508 213.377, 491

...

TopologyException: found non-noded intersection between LINESTRING (1 193, 0.00636508 35.012) and LINESTRING (4.6066 43.7196, 0 35) at 4.6065969428801292 43.719629927594525
TopologyException: found non-noded intersection between LINESTRING (378.664 326.173, 380.172 322.027) and LINESTRING (380.424 321.333, 379.433 324.059) at 379.69251145332476 323.34559350335678
TopologyException: side location conflict at 378.695652173913 326.17391304347825
Self-intersection at or near point 378.695652173913 326.17391304347825
TopologyException: side location conflict at 378.695652173913 326.17391304347825
Self-intersection at or near point 378.695652173913 326.17391304347825
TopologyException: side location conflict at 378.695652173913 326.17391304347825
Self-intersection at or near point 378.695652173913 326.17391304347825
TopologyException: found non-noded intersection between LINESTRING (378.662 326.178, 365 328) and LINESTRING (376.25 326.5, 376.25 326.5) at 376.24999999999989 326.5
Self-interse

TopologyException: no outgoing dirEdge found at 167 348
TopologyException: side location conflict at 78 356
TopologyException: EdgeRing::computePoints: found null Directed Edge
TopologyException: found non-noded intersection between LINESTRING (247.744 357.93, 247.744 357.93) and LINESTRING (250.152 357.834, 247.744 357.93) at 247.74418604651163 357.93023255813949
TopologyException: found non-noded intersection between LINESTRING (243.4 39.4, 243.4 39.4) and LINESTRING (243.4 39.4, 242.803 38.377) at 243.4000000020699 39.400000003548399
TopologyException: found non-noded intersection between LINESTRING (246 357.333, 246.579 357.977) and LINESTRING (247.744 357.93, 246.579 357.977) at 246.57915057915059 357.97683397683397
TopologyException: found non-noded intersection between LINESTRING (292 166, 281.409 71.6409) and LINESTRING (281.409 71.6409, 289 195) at 281.40866873065016 71.64086687306505
TopologyException: found non-noded intersection between LINESTRING (292 166, 281.409 71.6409)

TopologyException: found non-noded intersection between LINESTRING (151.423 465.19, 152.386 465.318) and LINESTRING (151.973 465.436, 153.5 465) at 152.38636363636363 465.31818181818181
TopologyException: found non-noded intersection between LINESTRING (151.423 465.19, 152.386 465.318) and LINESTRING (151.973 465.436, 153.5 465) at 152.38636363636363 465.31818181818181
TopologyException: found non-noded intersection between LINESTRING (151.423 465.19, 152.386 465.318) and LINESTRING (151.973 465.436, 153.5 465) at 152.38636363636363 465.31818181818181
TopologyException: found non-noded intersection between LINESTRING (195 470, 217.771 472.97) and LINESTRING (215 472, 210.333 472) at 210.33333333333334 472
TopologyException: found non-noded intersection between LINESTRING (118.954 405.594, 121.729 395.683) and LINESTRING (115.161 411.871, 119.342 404.205) at 119.34246575342463 404.20547945205482
TopologyException: found non-noded intersection between LINESTRING (195 470, 217.771 472.97)

TopologyException: found non-noded intersection between LINESTRING (120.5 405.75, 107.173 429.842) and LINESTRING (107.173 429.842, 107.173 429.842) at 107.17266187050356 429.84172661870514
TopologyException: found non-noded intersection between LINESTRING (168 352, 143.95 364.95) and LINESTRING (143.95 364.95, 155 363) at 143.95000000000005 364.94999999999999
TopologyException: found non-noded intersection between LINESTRING (98.6 400.6, 90 392) and LINESTRING (101 399, 98.6 400.6) at 98.600000000000009 400.60000000000002
TopologyException: found non-noded intersection between LINESTRING (168 352, 143.95 364.95) and LINESTRING (143.95 364.95, 155 363) at 143.95000000000005 364.94999999999999
TopologyException: found non-noded intersection between LINESTRING (98.6 400.6, 90 392) and LINESTRING (101 399, 98.6 400.6) at 98.600000000000009 400.60000000000002
TopologyException: found non-noded intersection between LINESTRING (168 352, 143.95 364.95) and LINESTRING (143.95 364.95, 155 363) 

IOError: File sample/5worker_rand6/PR_tbl_all.csv does not exist

### Run ML construction M step with the ground truth worker likelihoods to see if we can reconstruct T'

In [164]:
def runGTTileAdjacentMLConstruction(objid,workerErrorfunc,Qjfunc,A_percentile,DEBUG=False,PLOT_LIKELIHOOD=False,PLOT=False):
    '''
    Initilaize with majority vote tile , get good Qj estimates based on that
    Input:
    _____
    Tfunc : how to get ground truth
    workerErrorfunc : worker error model for computing p(ljk)
    Qjfunc : Model used for estimating Qj parameters
    objid,A_percentile

    Output:
    _____
    Tstar_idx_lst : list of Tstar index (all tiles that satisfy criterion) at every iteration
    Qj_lst : list of worker qualities at everystep
    Tstar_lst : Tstar at every step
    '''
    # ML Construction with E step as usual
    tiles = pkl.load(open(DATA_DIR+"/vtiles{}.pkl".format(objid)))
    indMat = pkl.load(open(DATA_DIR+"/indMat{}.pkl".format(objid)))
    workers = pkl.load(open(DATA_DIR+"/worker{}.pkl".format(objid)))

    tile_area = np.array(indMat[-1])

    Qj_lst=[]
    #if DEBUG: print "Coming up with T' combinations to search through"
    #Tprime_lst, Tprime_idx_lst = Tprimefunc(objid,tiles,indMat,fixedtopk=3, topk = 40,NTprimes=NTprimes)
    Tstar_lst = []
    Tstar_idx_lst =[]
    likelihood_lst=[]

    if DEBUG: print "Compute Core Tiles"
    Tstar,Tidx=core(tiles,indMat,1)
    Tstar_lst.append([Tstar])
    Tstar_idx_lst.append(Tidx)

    if DEBUG: print "Initialize Tiles"
    Tinit,Iidx=initT(tiles,indMat)
    if DEBUG: print "Estimate based on initial tileset and get good Qjs"
    if A_percentile!=-1:
        A_thres = np.percentile(tile_area,A_percentile)
    else:
        A_thres = np.median(tile_area[Iidx])
    # Get ground truth worker qualities 
    Qj = pd.read_csv(DATA_DIR+"GT_Qj.csv",index_col=0)
    Qj = Qj[Qj["object_id"]==objid]
    Qn1,Qn2,Qp1,Qp2= list(Qj["Qn1"]),list(Qj["Qn2"]),list(Qj["Qp1"]),list(Qj["Qp2"])
    
    #Run M step once
    plk=0
    if DEBUG: print "ML construction of Tstar"
    dPrime = 0

    exclude_idx = set(Tstar_idx_lst[0])
    Tidx_lst = list(exclude_idx)
    good_dPrime_tcount = len(exclude_idx)
    current_shell_tkidxs= Tidx
    past_shell_tkidxs= Tidx
    if DEBUG: print "Add core tiles to first occurence of tk satisfying criterion"
    Tstar_lst.append([Tstar_lst[0][0]])

    i=0
    while (good_dPrime_tcount!=0 or len(current_shell_tkidxs)!=0):
        ######
        if DEBUG:print "Excluding",exclude_idx
        current_shell_tkidxs = find_all_tk_in_shell(tiles,past_shell_tkidxs,list(exclude_idx))

        if DEBUG:
            print "d'={0}; good_dPrime_tcount={1}".format(dPrime,good_dPrime_tcount)
            print "Number of tks in shell: ",len(current_shell_tkidxs)
            print "Current shell index:",current_shell_tkidxs
        good_dPrime_tcount=0

        for k in current_shell_tkidxs:
            pInT = 0
            pNotInT = 0
            tk = tiles[k]
            # Compute pInT and pNotInT
            for j in range(len(workers)):
                ljk = indMat[j][k] #NOTE k doesn't correspond to k in tiles but in current_shell_tks so this is not good
                wid=workers[j]
                qp1 = Qp1[j]
                qp2 = Qp2[j]
                qn1 = Qn1[j]
                qn2 = Qn2[j]

                if tk.area>A_thres:
                    if ljk ==1:
                        if qp1!=-1:
                            pInT+=np.log(qp1)
                        if qn1!=-1:
                            pNotInT+=np.log(1-qn1)
                    else:
                        if qp1!=-1:
                            pInT+=np.log(1-qp1)
                        if qn1!=-1:
                            pNotInT+=np.log(qn1)
                else:
                    if ljk ==1:
                        if qp2!=-1:
                            pInT+=np.log(qp2)
                        if qn2!=-1:
                            pNotInT+=np.log(1-qn2)
                    else:
                        if qp2!=-1:
                            pInT+=np.log(1-qp2)
                        if qn2!=-1:
                            pNotInT+=np.log(qn2)
            # Check if tk satisfy constraint
            if pInT<pNotInT:
                plk+=pNotInT
            elif pInT>=pNotInT:
                plk+=pInT
                # if satisfy criterion, then add to Tstar
                good_dPrime_tcount+=1
                if DEBUG: print "Adding tk",k
                try:
                    Tstar_lst[i]=[Tstar_lst[i][0].union(tk)]
                    Tidx_lst.append(k)
                except(shapely.geos.TopologicalError):
                    try:
                        Tstar_lst[i]=[Tstar_lst[i][0].buffer(0).union(tk.buffer(-1e-10))]
                        Tidx_lst.append(k)
                    except(shapely.geos.TopologicalError):
                        try:
                            Tstar_lst[i]=[Tstar_lst[i][0].buffer(-1e-10).union(tk)]
                            Tidx_lst.append(k)
                        except(shapely.geos.TopologicalError):
                            try:
                                Tstar_lst[i]=[Tstar_lst[i][0].buffer(-1e-10).union(tk.buffer(-1e-10))]
                                Tidx_lst.append(k)
                            except(shapely.geos.TopologicalError):
                                try:
                                    Tstar_lst[i]=[Tstar_lst[i][0].union(tk.buffer(1e-10))]
                                    Tidx_lst.append(k)
                                except(shapely.geos.TopologicalError):
                                    try:
                                        Tstar_lst[i]=[Tstar_lst[i][0].buffer(1e-10).union(tk)]
                                        Tidx_lst.append(k)
                                    except(shapely.geos.TopologicalError):
                                        try:
                                            Tstar_lst[i]=[Tstar_lst[i][0].buffer(1e-10).union(tk.buffer(1e-10))]
                                            Tidx_lst.append(k)
                                        except(shapely.geos.TopologicalError):
                                            print "Shapely Topological Error: unable to add tk, Tstar unchanged; at k=",k
                                            pkl.dump(Tstar_lst[i][0],open("problematic_Tstar_{0}.pkl".format(k),'w'))
                                            pkl.dump(tk,open("problematic_tk_{0}.pkl".format(k),'w'))
                                            pass

        ############################################################################################################
        if PLOT:
            plt.figure()
            for c in current_shell_tkidxs:plot_coords(tiles[c],color="red",fill_color="red") #current front
            for c in past_shell_tkidxs:plot_coords(tiles[c],color="cyan",linewidth=5,linestyle='--') #past front
            for c in exclude_idx:plot_coords(tiles[c],color="gray",fill_color="gray")#excluded coord
            plot_coords(Tstar_lst[i][0],linestyle="--",linewidth=2,color="blue")#current Tstar
            for c in Tidx_lst:plot_coords(tiles[c],linewidth=2,color="green",fill_color="green")#new Tstar
            plt.ylim(40,100)
        #Updates
        Tstar = Tstar_lst[i][0].buffer(0)
        dPrime+=1
        past_shell_tkidxs= current_shell_tkidxs
        exclude_idx= exclude_idx.union(current_shell_tkidxs)
    #Storage
    Tstar_idx_lst.append(Tidx_lst)
    likelihood_lst.append(plk)

    return Tstar_idx_lst , likelihood_lst, Tstar_lst

In [166]:
Nworker=5
batch_num=0

DATA_DIR = "sample/{0}worker_rand{1}/".format(Nworker,batch_num)

In [167]:
reconstructed_Plst = []
reconstructed_Rlst = []
for objid in object_lst:
    tiles = pkl.load(open(DATA_DIR+"/vtiles{}.pkl".format(objid)))
    Tstar_idx_lst ,likelihood_lst,Tstar_lst=runGTTileAdjacentMLConstruction(objid,workerErrorfunc="GTLSA",Qjfunc=QjGTLSA,A_percentile=-1,DEBUG=False,PLOT_LIKELIHOOD=False)
    P,R = compute_PR(objid,Tstar_idx_lst[-1],tiles)
    reconstructed_Plst.append(P)
    reconstructed_Rlst.append(R)

TopologyException: side location conflict at 367.12660718971404 209.43059564418817
TopologyException: found non-noded intersection between LINESTRING (196 323, 198.6 318.45) and LINESTRING (198.6 318.45, 194.556 323) at 198.59999999999999 318.44999999999999
Self-intersection at or near point 198.60000000000002 318.44999999999993
TopologyException: found non-noded intersection between LINESTRING (196 323, 198.6 318.45) and LINESTRING (198.6 318.45, 194.556 323) at 198.59999999999999 318.44999999999999
Self-intersection at or near point 198.60000000000002 318.44999999999993
TopologyException: found non-noded intersection between LINESTRING (196 323, 198.6 318.45) and LINESTRING (198.6 318.45, 194.556 323) at 198.59999999999999 318.44999999999999
Self-intersection at or near point 198.60000000000002 318.44999999999993
TopologyException: found non-noded intersection between LINESTRING (196 323, 198.6 318.45) and LINESTRING (198.6 318.45, 194.556 323) at 198.59999999999999 318.4499999999999

Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310 154.99999999999994
Self-intersection at or near point 310 154.99999999999994
TopologyException: found non-noded intersection between LINESTRING (312 151, 310 155) and LINESTRING (318 147, 310 155) at 310

TopologyException: found non-noded intersection between LINESTRING (268.396 387.089, 270 378) and LINESTRING (268.938 384.021, 268 386) at 268.9375 384.02083333333331
TopologyException: found non-noded intersection between LINESTRING (246.344 397.213, 246.344 397.213) and LINESTRING (267 395, 246.344 397.213) at 246.34426229508199 397.21311475409834
TopologyException: found non-noded intersection between LINESTRING (243.073 397.564, 246.344 397.213) and LINESTRING (243.073 397.564, 238 397) at 243.07272727272726 397.56363636363636
TopologyException: found non-noded intersection between LINESTRING (415.965 255.101, 419.897 270.365) and LINESTRING (415.965 255.101, 417.939 262.765) at 416.84027363340283 258.49755035246847
TopologyException: found non-noded intersection between LINESTRING (415.965 255.101, 417.939 262.765) and LINESTRING (415.965 255.101, 419.897 270.365) at 416.84027363340283 258.49755035246847
TopologyException: found non-noded intersection between LINESTRING (391 55, 3

Self-intersection at or near point 360.40000000000003 78.199999999999889
TopologyException: found non-noded intersection between LINESTRING (357 85, 360.4 78.2) and LINESTRING (368 63, 360.4 78.2) at 360.40000000000003 78.199999999999918
Self-intersection at or near point 360.40000000000003 78.199999999999889
TopologyException: found non-noded intersection between LINESTRING (357 85, 360.4 78.2) and LINESTRING (368 63, 360.4 78.2) at 360.40000000000003 78.199999999999918
Self-intersection at or near point 360.40000000000003 78.199999999999889
TopologyException: found non-noded intersection between LINESTRING (357 85, 360.4 78.2) and LINESTRING (368 63, 360.4 78.2) at 360.40000000000003 78.199999999999918
Self-intersection at or near point 360.40000000000003 78.199999999999889
TopologyException: found non-noded intersection between LINESTRING (357 85, 360.4 78.2) and LINESTRING (368 63, 360.4 78.2) at 360.40000000000003 78.199999999999918
Self-intersection at or near point 360.400000000

TopologyException: found non-noded intersection between LINESTRING (96.3265 111.306, 96.3265 111.306) and LINESTRING (90.2381 116.786, 96.3265 111.306) at 96.326530612244881 111.30612244897959
TopologyException: found non-noded intersection between LINESTRING (297.778 322.556, 295.882 327.294) and LINESTRING (295.882 327.294, 300 322) at 295.88235294117652 327.29411764705867
TopologyException: side location conflict at 300 317
TopologyException: found non-noded intersection between LINESTRING (327 154, 337.648 155.852) and LINESTRING (337.648 155.852, 327 155) at 337.64814814814798 155.85185185185185
TopologyException: found non-noded intersection between LINESTRING (180.984 289.145, 180.984 289.145) and LINESTRING (180.984 289.145, 180.984 289.145) at 180.9835526315789 289.14473684210526
TopologyException: found non-noded intersection between LINESTRING (307 116, 302.842 107.684) and LINESTRING (302.842 107.684, 306 111) at 302.84210526315798 107.68421052631589
TopologyException: foun

Self-intersection at or near point 378.695652173913 326.17391304347825
TopologyException: found non-noded intersection between LINESTRING (378.662 326.178, 379.534 326.062) and LINESTRING (378.696 326.174, 378.664 326.173) at 378.695652173913 326.17391304347825
Self-intersection at or near point 378.695652173913 326.17391304347825
TopologyException: found non-noded intersection between LINESTRING (246 358, 270.372 357.025) and LINESTRING (265.959 357.202, 270.385 357.025) at 269.93016895011095 357.04279324199553
TopologyException: found non-noded intersection between LINESTRING (241 347, 243.632 352.789) and LINESTRING (243.286 352.357, 243.632 352.789) at 243.63157894736835 352.78947368421041
TopologyException: found non-noded intersection between LINESTRING (274.333 349, 274.333 349) and LINESTRING (274.333 349, 274.333 349) at 274.33333333345797 348.99999999850399
TopologyException: side location conflict at 78 356
TopologyException: found non-noded intersection between LINESTRING (

In [168]:
reconstructed_Plst

[0.19513802759554058,
 0.961283940079857,
 0.9469620256682164,
 0.9708669778056955,
 0.9860657803210537,
 0.969954952987154,
 0.9458439942556413,
 0.9621995698211235,
 0.9968547314131838,
 0.9197641811099027,
 0.9249211769811901,
 0.9016594172616282,
 0.9786404369282409,
 0.9543111494377815,
 0.8693160276952182,
 0.9338020834496665,
 0.9222715823799703,
 0.9825949185360743,
 0.9637652995650068,
 0.9767989016821484,
 0.9923756399916978,
 0.7725277202269896,
 0.9486240221129278,
 0.9693443256538107,
 0.943105085593935,
 0.9730181624714889,
 0.9798624976717247,
 0.9336883317582926,
 0.9733831257000037,
 0.8491239307903167,
 0.9727029596166933,
 0.997730077990624,
 0.9880239130139529,
 0.9849365415157119,
 0.17010095279042073,
 0.9856023264721963,
 0.945380980462112,
 0.9922378784879062,
 0.9878806074245543,
 0.003500599930621462,
 0.050523627958767456,
 0.9612872353112322,
 0.9674302889415317,
 0.9542232637818413,
 0.950455334591807,
 0.9560742501514032,
 0.9777439616951173]

In [170]:
GT_PR = pd.read_csv(DATA_DIR+"GT_PR.csv",index_col=0)

In [171]:
GT_PR["Reconstructed P"]=reconstructed_Plst
GT_PR["Reconstructed R"]=reconstructed_Rlst

In [172]:
GT_PR

,object_id,GT Tile-based Precision,GT Tile-based Recall,GT A_thres,Reconstructed P,Reconstructed R
0,1,0.199121,0.968257,8.336538,0.195138,0.942669
1,2,0.962979,0.981210,11.607561,0.961284,0.951286
2,3,0.939696,0.095305,12.099832,0.946962,0.922782
3,4,0.969633,0.942835,12.649492,0.970867,0.888187
4,5,0.951914,0.932064,9.465079,0.986066,0.842963
5,6,0.961495,0.900493,1.208874,0.969955,0.824487
6,7,0.949913,0.582125,13.908456,0.945844,0.561117
7,8,0.966967,0.928553,22.750652,0.962200,0.709340
8,9,0.986355,0.965070,8.486176,0.996855,0.909754
9,10,0.984834,0.976777,11.647102,0.919764,0.933538


In [173]:
GT_PR.to_csv("GTreconstruction_PR_experiment.csv")